In [ ]:
# Import all needed Modules for future use

import time
import random
from collections import Counter, defaultdict
import math

In [ ]:
# Load the list of 5 letter words
# Word list from https://gist.github.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b

def load_word_list(filepath="wordle-answers-alphabetical.txt"):
    with open(filepath, "r") as file:
        words = [line.strip().lower() for line in file if len(line.strip()) == 5]
    return words

wordList = load_word_list()


random.seed(42)  # Preset Seed for replicable results
wordList = random.sample(wordList, 500)

print(wordList)
print(len(wordList))

['craft', 'ardor', 'level', 'humph', 'grind', 'donor', 'clout', 'water', 'chaos', 'shire', 'await', 'audio', 'chose', 'grain', 'harem', 'titan', 'arose', 'frond', 'watch', 'sheep', 'grate', 'sonar', 'lithe', 'adorn', 'empty', 'shoal', 'pence', 'lipid', 'eight', 'golly', 'paste', 'cloud', 'choir', 'rebel', 'claim', 'primo', 'picky', 'knife', 'being', 'spool', 'verve', 'cynic', 'reach', 'cadet', 'whoop', 'march', 'proud', 'foist', 'broke', 'beset', 'guide', 'maize', 'cameo', 'harpy', 'cloak', 'woody', 'worry', 'speed', 'pupal', 'equal', 'quest', 'posit', 'given', 'lager', 'brute', 'false', 'vapor', 'humor', 'erect', 'squat', 'rearm', 'laugh', 'graph', 'oddly', 'blown', 'gulch', 'awake', 'needy', 'saner', 'lapel', 'breed', 'glaze', 'nasty', 'globe', 'thief', 'rowdy', 'spoof', 'drank', 'knock', 'yield', 'hutch', 'visit', 'kitty', 'silky', 'salad', 'prove', 'grape', 'dodge', 'touch', 'teach', 'child', 'bicep', 'coven', 'earth', 'endow', 'shirk', 'brain', 'reply', 'refer', 'steam', 'unwed', 

In [ ]:
# Feedback for a guess

def get_feedback(guess, target):
    """
    Returns feedback for a Wordle guess:
    - 'g' for green (correct letter and position)
    - 'y' for yellow (correct letter, wrong position)
    - 'b' for black/gray (letter not in word)
    """
    feedback = ['b'] * 5
    target_chars = list(target)
    used_indices = []

    # First pass for greens
    for i in range(5):
        if guess[i] == target[i]:
            feedback[i] = 'g'
            target_chars[i] = None
            used_indices.append(i)

    # Second pass for yellows
    for i in range(5):
        if feedback[i] == 'b' and guess[i] in target_chars:
            index = target_chars.index(guess[i])
            feedback[i] = 'y'
            target_chars[index] = None

    return ''.join(feedback)

In [ ]:
# Creates a list of calculated entropies of all words without guesses
# Useful for future performance of the Entropy wordle solver


def precompute_first_entropy(wordList):
    entropy_scores = [(word, calculate_entropy(word, wordList)) for word in wordList]
    entropy_scores.sort(key=lambda x: x[1], reverse=True)
    return entropy_scores

precomputed_entropy = precompute_first_entropy(wordList)


In [ ]:
def update_possible_words(possible_words, guess, feedback):
    updated_words = []

    for word in possible_words:
        if get_feedback(guess, word) == feedback:
            updated_words.append(word)

    return updated_words


This is the rule based solver approach. Just like other basic rule solvers this simply evluates words and removes words from the possible word list taht do not have the letters that are in the feedback of the guess.

In [ ]:
def rule_based_solver(target, word_list):
    # Common Starting guess for players
    guess = "slate"
    attempts = 1
    candidates = word_list.copy()

    while guess != target:

        feedback = get_feedback(guess, target)

        # Filter words based on feedback
        new_candidates = []
        for word in candidates:
            if get_feedback(guess, word) == feedback:
                new_candidates.append(word)
        candidates = new_candidates

        # Stop if no words left
        if not candidates or attempts > 6:
            return attempts, False  # Failure case

        guess = candidates[0]
        attempts += 1

    return attempts, True  # Success

In [ ]:
def evaluate_rule_solver(word_list):
    total_attempts = 0
    solved = 0

    for target in word_list:
        attempts, success = rule_based_solver(target, word_list)
        total_attempts += attempts
        if success:
            solved += 1

    average_attempts = total_attempts / len(word_list)
    success_rate = solved / len(word_list) * 100

    print(f"\nAverage attempts: {average_attempts:.2f}")
    print(f"Success rate: {success_rate:.2f}%")

    return average_attempts, success_rate


In [ ]:
def letter_frequency(words):
    freq = [Counter() for _ in range(5)]
    for word in words:
        for i, char in enumerate(word):
            freq[i][char] += 1
    return freq


In [ ]:
def score_word(word, freq_table):
    score = 0
    seen = set()
    for i, char in enumerate(word):
        if char not in seen:
            score += freq_table[i][char]
            seen.add(char)
    return score


In [ ]:
def frequency_based_solver(target, word_list):
    possible_words = word_list[:]
    attempts = 0

    while attempts < 6:
        freq_table = letter_frequency(possible_words)
        scored_words = [(word, score_word(word, freq_table)) for word in possible_words]
        scored_words.sort(key=lambda x: x[1], reverse=True)

        guess = scored_words[0][0]
        feedback = get_feedback(guess, target)
        attempts += 1

        if feedback == 'g' * 5:
            return attempts, True

        possible_words = update_possible_words(possible_words, guess, feedback)

    return attempts, False


In [ ]:
def evaluate_freq_solver(word_list):
    total_attempts = 0
    solved = 0

    for target in word_list:
        attempts, success = frequency_based_solver(target, word_list)
        total_attempts += attempts
        if success:
            solved += 1

    average_attempts = total_attempts / len(word_list)
    success_rate = solved / len(word_list) * 100

    print(f"\nAverage attempts: {average_attempts:}")
    print(f"Success rate: {success_rate:}%")

    return average_attempts, success_rate


In [ ]:
def calculate_entropy(guess, possible_words):
    feedback_counts = defaultdict(int)

    for word in possible_words:
        feedback = tuple(get_feedback(guess, word))
        feedback_counts[feedback] += 1

    total = len(possible_words)
    entropy = 0.0
    for count in feedback_counts.values():
        p = count / total
        entropy -= p * math.log2(p)

    return entropy

def entropy_based_solver(word_list, target_word, precomputed_entropy=None):
    possible_words = word_list.copy()
    attempts = 0
    success = False



    while attempts < 6:
        if attempts == 0 and precomputed_entropy is not None:
            entropy_scores = precomputed_entropy
        else:
            entropy_scores = [(word, calculate_entropy(word, possible_words)) for word in possible_words]
            entropy_scores.sort(key=lambda x: x[1], reverse=True)

        guess = entropy_scores[0][0]
        feedback = get_feedback(guess, target_word)
        print(f"Guess: {guess}, Feedback: {feedback}")
        attempts += 1

        if guess == target_word:
            success = True
            break

        possible_words = update_possible_words(possible_words, guess, feedback)

    return attempts, success


In [ ]:
def evaluate_entropy_solver(word_list):
    total_attempts = 0
    solved = 0

    for target in word_list:
        attempts, success = entropy_based_solver(word_list, target)
        total_attempts += attempts
        if success:
            solved += 1

    average_attempts = total_attempts / len(word_list)
    success_rate = solved / len(word_list) * 100

    print(f"\nAverage attempts: {average_attempts:}")
    print(f"Success rate: {success_rate:}%")

    return average_attempts, success_rate


In [ ]:
start = time.time()
evaluate_rule_solver(wordList)
end = time.time()

print(f"Execution time: {end - start:} seconds")


Average attempts: 3.16
Success rate: 100.00%
Execution time: 0.5053036212921143 seconds


In [ ]:
start = time.time()
evaluate_freq_solver(wordList)
end = time.time()

print(f"Execution time: {end - start:} seconds")


Average attempts: 3.09
Success rate: 100.00%
Execution time: 1.185394287109375 seconds


In [ ]:
start = time.time()
evaluate_entropy_solver(wordList)
end = time.time()

print(f"Execution time: {end - start:} seconds")


Average attempts: 2.98
Success rate: 100.00%
Execution time: 309.7014527320862 seconds
